Le but de ce notebook est de comparer la performance (basée sur la métrique RMSE) de nos algorithmes lorsqu'on prend des matrices aléatoires de même taille et de même rang et qu'on'change le nombre d'entrées révélées.

# Importation des modules nécessaires

In [1]:
import numpy as np
import numpy.linalg as alg
from utilities import generate_low_rank, generate_entries
from convex_optimization import test_convex
from gradient_descent_script import generate_prediction_noprint, generate_M
from clustering_method import complete_matrix
from clean_altmin import predict_AltMin
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
import warnings

warnings.filterwarnings("ignore")


# Variables et fonctions pour tester chaque algorithme

In [3]:
n=100 #taille des matrices
r=int(n**(1/5)) #rang des matrices (ce rang est considéré comme assez faible pour la méthode d'optimisation convexe)
step=150 #pas sur le nombre d'entrées
nb_iter=1 #nombre d'expériences pour chaque valeur du nombre d'entrées

In [4]:
test_gradient=lambda A,omega: alg.norm(A-generate_prediction_noprint(A,omega,learning_rate=0.1))/A.size**0.5
def test_alternating(A,omega):
    M=generate_M(A,omega)
    P=predict_AltMin(M,*A.shape)
    return alg.norm(A-P)/P.size**0.5
def test_clustering(A,omega): 
    n,p=A.shape
    M=np.zeros((n,p))
    for i,j in omega:
        M[i,j]=A[i,j]
    return alg.norm(A-complete_matrix(M))/A.size**0.5

Chacune de ces fonctions prend en entrée la matrice complète et l'ensemble omega des entrées révélées et retourne la RMSE de la prédiction de l'algorithme correspondant.

# Traçage des courbes

In [5]:
nb_samples=np.arange(1,3000,step)
RMSE_convex=np.ndarray(nb_samples.shape)
RMSE_gradient=np.ndarray(nb_samples.shape)
RMSE_alternating=np.ndarray(nb_samples.shape)
RMSE_clustering=np.ndarray(nb_samples.shape)
for p in tqdm(range(len(nb_samples))):
    k=nb_samples[p] #nombre d'entrées
    #listes contenant les valeurs de la RMSE obtenue pour chaque expérience
    X_convex=np.ndarray(nb_iter)
    X_gradient=np.ndarray(nb_iter)
    X_alternating=np.ndarray(nb_iter)
    X_clustering=np.ndarray(nb_iter)
    for i in range(nb_iter):
        omega=generate_entries(n,n,k)
        A=generate_low_rank(n,n,r)
        X_convex[i]=test_convex(A,omega)
        X_gradient[i]=test_gradient(A,omega)
        X_clustering[i]=test_clustering(A,omega)
        X_alternating[i]=test_alternating(A,omega)
    #On calcule la moyenne de la RMSE
    RMSE_convex[p]=np.mean(X_convex)
    RMSE_gradient[p]=np.mean(X_gradient)
    RMSE_alternating[p]=np.mean(X_alternating)
    RMSE_clustering[p]=np.mean(X_clustering)





  0%|          | 0/20 [00:00<?, ?it/s]

 10%|█         | 2/20 [02:23<25:03, 83.53s/it]

In [ ]:
plt.plot(nb_samples,RMSE_convex,label="Optimisation convexe")
plt.plot(nb_samples,RMSE_gradient,label="Descente du gradient")
plt.plot(nb_samples,RMSE_alternating,label="Minimisation alternée")
plt.plot(nb_samples,RMSE_clustering,label="Clustering")
plt.ylabel("RMSE moyen")
plt.xlabel("nombre d'entrées révélées")
plt.title("RMSE en fonction du nombre d'entrées révélées")
plt.legend()
plt.show()